[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-3/breakpoints.ipynb) [![在 LangChain Academy 中打开](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239469-lesson-2-breakpoints)


# 断点

## 回顾

在 `human-in-the-loop`（人类在环）场景下，我们常常希望在图运行时查看它的输出。

我们已经通过流式传输为此打下了基础。

## 目标

现在，让我们讨论一下 `human-in-the-loop` 的动机：

(1) `Approval`（批准）- 我们可以中断智能体，向用户展示状态，并允许用户接受某个动作

(2) `Debugging`（调试）- 我们可以回滚图，以复现或规避问题

(3) `Editing`（编辑）- 你可以修改状态

LangGraph 提供了多种方式来获取或更新智能体状态，以支持不同的 `human-in-the-loop` 工作流。

首先，我们将介绍[断点](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/breakpoints/#simple-usage)，它提供了一种在特定步骤停止图的简单方法。

之后我们会展示这如何实现用户 `approval`（批准）。


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk langgraph-prebuilt

In [ ]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("OPENAI_API_KEY")
_set_env("DASHSCOPE_API_KEY")

## 面向人类批准的断点

重新回顾我们在模块 1 中使用过的那个简单智能体。

假设我们担心工具的使用：我们希望在智能体调用任何工具之前先获得批准。

我们只需要在编译图时传入 `interrupt_before=["tools"]`，其中 `tools` 是我们的工具节点。

这意味着执行会在节点 `tools` 之前被中断，也就是执行工具调用之前。


In [3]:
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi


def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a by b.

    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add, multiply, divide]
# llm = ChatOpenAI(model="gpt-4o")
llm = ChatTongyi(model="qwen-plus")
llm_with_tools = llm.bind_tools(tools)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
)


# Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# REACT模式
# Define edges: these determine the control flow
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["tools"], checkpointer=memory)

# Show
# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [6]:
# Input
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_0ea5704deedb4799b68e63)
 Call ID: call_0ea5704deedb4799b68e63
  Args:
    a: 2
    b: 3


我们可以获取状态并查看下一步将要调用的节点。

这是验证图已被中断的好方法。


In [7]:
state = graph.get_state(thread)
state.next

('tools',)

现在介绍一个不错的小技巧。

当我们以 `None` 调用图时，它会直接从上一次状态检查点继续执行！

![breakpoints.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbae7985b747dfed67775d_breakpoints1.png)

为了更清晰，LangGraph 会重新发出当前状态，其中包含带有工具调用的 `AIMessage`。

接着它会执行图中的后续步骤，从工具节点开始。

我们可以看到工具节点使用这个工具调用运行，然后再把结果传回聊天模型，生成最终回答。


In [8]:
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  multiply (call_0ea5704deedb4799b68e63)
 Call ID: call_0ea5704deedb4799b68e63
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


现在，我们把这些和一个明确的用户批准步骤结合起来，该步骤会接收用户输入。


In [9]:
# Input
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}

# Thread
thread = {"configurable": {"thread_id": "2"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Get user feedback
user_approval = input("Do you want to call the tool? (yes/no): ")

# Check approval
if user_approval.lower() == "yes":

    # If approved, continue the graph execution
    for event in graph.stream(None, thread, stream_mode="values"):
        event["messages"][-1].pretty_print()

else:
    print("Operation cancelled by user.")

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_9b921ac23e1145908167c6)
 Call ID: call_9b921ac23e1145908167c6
  Args:
    a: 2
    b: 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_9b921ac23e1145908167c6)
 Call ID: call_9b921ac23e1145908167c6
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


### 在 LangGraph API 中使用断点

**⚠️ 免责声明**

自从录制这些视频以来，我们已经更新了 Studio，使其可以在本地运行并在浏览器中打开。现在推荐的方式是以这种形式运行 Studio（而不是像视频中展示的桌面应用）。关于本地开发服务器请查看[这里](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)的文档，关于本地 Studio 的运行方式请查看[这里](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。在本模块的 `/studio` 目录中，在终端运行以下命令即可启动本地开发服务器：

```
langgraph dev
```

你应该会看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中访问 Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。

LangGraph API [支持断点](https://langchain-ai.github.io/langgraph/cloud/how-tos/human_in_the_loop_breakpoint/#sdk-initialization)。


In [ ]:
if "google.colab" in str(get_ipython()):
    raise Exception(
        "Unfortunately LangGraph Studio is currently not supported on Google Colab"
    )

In [10]:
# This is the URL of the local development server
from langgraph_sdk import get_client

client = get_client(url="http://127.0.0.1:2024")

如上所示，我们可以在编译运行于 Studio 的图时添加 `interrupt_before=["node"]`。

不过，在使用 API 时，你也可以直接把 `interrupt_before` 传给 `stream` 方法。


In [11]:
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=initial_input,
    stream_mode="values",
    interrupt_before=["tools"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get("messages", [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'Multiply 2 and 3', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '1743058b-054d-4c26-8b99-34fd4b6d12aa', 'example': False}
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_6d092e97a10a47748ce2c6', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '1946d8b0-dc5c-4ac2-a2e5-2c996a889e32', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 0}}}, 'type': 'ai', 'name': None, 'id': 'run--4ca93d06-8d1f-4e4d-b797-0d4004e95e1b-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a'

然后，就像我们之前那样，通过传入 `thread_id` 和 `None` 作为输入，就可以从断点继续执行！


In [12]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=None,
    stream_mode="values",
    interrupt_before=["tools"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get("messages", [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_6d092e97a10a47748ce2c6', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '1946d8b0-dc5c-4ac2-a2e5-2c996a889e32', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 0}}}, 'type': 'ai', 'name': None, 'id': 'run--4ca93d06-8d1f-4e4d-b797-0d4004e95e1b-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_6d092e97a10a47748ce2c6', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
--------------------------------------------------
Receiving new event of type: values...
{'content': '6', 'additional_kwargs': {}, 'response_m